In [ ]:
import os
code_path = os.getcwd()
data_path = code_path + 'Original X-ray Image Dataset'
covid_path = data_path + '/covid/'
none_path = data_path + '/none/'
pneumonia_path = data_path + '/pneumonia/'
print(len(os.listdir(covid_path))) #125
print(len(os.listdir(none_path))) #500
print(len(os.listdir(pneumonia_path))) #500

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, losses, optimizers, utils, regularizers
from tensorflow.keras import backend as K
from livelossplot import PlotLossesKeras
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import glob
from sklearn.utils import class_weight
import random

In [ ]:
covid = glob.glob(covid_path+'*.*')
pneumonia = glob.glob(pneumonia_path+'*.*')
none = glob.glob(none_path+'*.*')

data = []
labels = []

def image_preprocessing(i):
  image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
  target_size= (256,256))
  image=np.asarray(image)
  image = image.astype('float32')/255
  data.append(image)

for img in covid:
  image_preprocessing(img)
  labels.append(0)
for img in none:
  image_preprocessing(img)
  labels.append(1)
for img in pneumonia:
  image_preprocessing(img)
  labels.append(2)

data = np.array(data)
labels = np.array(labels)

In [ ]:
def makemodel(k):
      global model
      K.clear_session()
      model_input = layers.Input(shape = (256,256,3))
      x = layers.Conv2D(8, 3, 1, padding='same', use_bias=False)(model_input)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(16, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(32, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(16, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(32, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(64, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(32, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(64, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(128, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(64, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(128, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(128, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(128, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(k, k, 1, padding="same")(x)
      x = layers.ReLU()(x)
      x = layers.BatchNormalization()(x)
      x_ls = layers.Flatten()(x)

      "orthogonal constraint"
      ls_split_dim = k
      x_ls_norm = layers.Lambda(lambda t: K.l2_normalize(t, axis=1))(x_ls)
      ls_split = layers.Lambda(lambda tensor: tf.split(tensor, [int(K.int_shape(x_ls)[1]/ls_split_dim)]*ls_split_dim,axis=1))(x_ls_norm)
      ls_stack = layers.Lambda(lambda tensor: tf.stack(tensor, axis=2))(ls_split)
      ls_utu = layers.Lambda(lambda tensor: tf.linalg.matmul(tensor,tensor,transpose_a=True))(ls_stack)
      ls_utu_diag = layers.Lambda(lambda tensor: tf.compat.v1.matrix_diag_part(tensor)-1)(ls_utu)
      ls_utu2 = layers.Lambda(lambda tensor: tf.compat.v1.matrix_set_diag(tensor,ls_utu_diag))(ls_utu)
      l_orths = layers.Lambda(lambda tensor: K.sqrt(K.sum(K.square(tensor),axis=[1,2])))(ls_utu2)
      l_orths = layers.Reshape((1,))(l_orths)
      model_output = layers.Dense(3, activation="softmax")(x_ls)
      model = Model(model_input, [model_output, l_orths])
      return model

In [ ]:
def train_model(loss_weight,k):
    BS = 32
    
    seed = 7
    numpy.random.seed(seed)
    X = data
    y = labels
    fold = 1
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    for train, test in kfold.split(X,y):
        makemodel(k)
        model.summary()

        initial_learning_rate = 3e-3
        decay_steps = 500
        decay_rate = 0.6
        learning_rate_fn = tf.keras.optimizers.schedules.ExponentialDecay(
          initial_learning_rate, decay_steps, decay_rate)

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_fn),
                  loss= ['categorical_crossentropy', 'mean_squared_error'],
                  metrics=['accuracy'],
                  loss_weights = [loss_weight, 1-loss_weight])

        y[train] = tf.keras.utils.to_categorical(y[train], 3)
        y[test] = tf.keras.utils.to_categorical(y[test], 3)
        X_train_zeros = np.zeros((900,1))
        X_test_zeros = np.zeros((225))

        sampleweights = sklearn.utils.class_weight.compute_sample_weight('balanced', y[train])
        history = model.fit(X[train], [y[train], X_train_zeros],
                                batch_size = BS,
                                epochs = 100,
                                validation_data = (X[test], [y[test], X_test_zeros]),
                                callbacks=[PlotLossesKeras()],
                                sample_weight = [sampleweights, sampleweights])
        print(history.history.keys())
        
        model.save(data_path+'OS_3class_model_'+fold+'.h5')
        model.save_weights(data_path+'OS_3class_modelweights_'+fold+'.h5')

        test_set = X[test]
        predIdxs, _ = model.predict(test_set,
          batch_size=BS)
        predIdxs = np.argmax(predIdxs, axis=1)
        y[test] = np.argmax(y[test], axis=1)

        def plot_cm(y_true, y_pred, figsize=(10,10)):
            cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
            cm_sum = np.sum(cm, axis=1, keepdims=True)
            cm_perc = cm / cm_sum.astype(float) * 100
            annot = np.empty_like(cm).astype(str)
            nrows, ncols = cm.shape
            for i in range(nrows):
                for j in range(ncols):
                    c = cm[i, j]
                    p = cm_perc[i, j]
                    if i == j:
                        s = cm_sum[i]
                        annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
                    elif c == 0:
                        annot[i, j] = ''
                    else:
                        annot[i, j] = '%.1f%%\n%d' % (p, c)
            cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
            cm.index.name = 'Actual'
            cm.columns.name = 'Predicted'
            fig, ax = plt.subplots(figsize=figsize)
            sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)
            plt.savefig(data_path+'OS_3class_cm_'+fold+'.svg')
            plt.show()
        plot_cm(y[test], predIdxs,x)

        names = ['COVID','pneumonia','none']
        print('Classification Report')
        print(classification_report(y[test], predIdxs,
          target_names=names))
        clsf_report = pd.DataFrame(classification_report(y[test], predIdxs,
          target_names=names, output_dict=True)).transpose()
        clsf_report.to_csv(data_path + 'OS_3class_report_'+fold+'.csv', index= True)

        fold+=1

train_model(0.8,3)